# ETL Project

In [21]:
# Importing Dependencies

import pandas as pd
import numpy as np
import pymongo
import os

### 1. Extracting data

The main source where data was retrieved is Kaggle at the dataset <a href="https://www.kaggle.com/rsrishav/youtube-trending-video-dataset"> Youtube Trending Video Dataset </a>. This is updated Daily at Kaggle.

In [20]:
# Importing downloaded data 

base_path= "Resources/"
file_list = os.listdir(base_path)
dF = {}
country = []


for file in file_list:
    if file[-4:] == ".csv":
        country.append(file[0:2])
        dF[country[len(country)-1]] = pd.read_csv(f'{base_path}/{file}', encoding="latin-1")
        
dF.keys()

dict_keys(['CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU', 'US'])

In [3]:
# Way of seeing the items inside the dictionary
df_list = list(dF.items())
df_list

[('CA',
            video_id trending_date  \
  0      n1WpP7iowLc      17.14.11   
  1      0dBIkQ4Mz1M      17.14.11   
  2      5qpjK5DgCt4      17.14.11   
  3      d380meD0W0M      17.14.11   
  4      2Vv-BfVoq4g      17.14.11   
  ...            ...           ...   
  40876  sGolxsMSGfQ      18.14.06   
  40877  8HNuRNi8t70      18.14.06   
  40878  GWlKEM3m2EE      18.14.06   
  40879  lbMKLzQ4cNQ      18.14.06   
  40880  POTgw38-m58      18.14.06   
  
                                                     title  \
  0            Eminem - Walk On Water (Audio) ft. BeyoncÃ©   
  1                          PLUSH - Bad Unboxing Fan Mail   
  2      Racist Superman | Rudy Mancuso, King Bach & Le...   
  3                               I Dare You: GOING BALD!?   
  4            Ed Sheeran - Perfect (Official Music Video)   
  ...                                                  ...   
  40876                       HOW2: How to Solve a Mystery   
  40877                   Eli Lik Lik

### 2. Transforming the data into the main DataFrames to use

In [4]:
# Merging all of the dataframes using append, and adding new columns with the country

df_list[0][1]['country'] = df_list[0][0]
all_countries = df_list[0][1]
del df_list[0]
for data in df_list:
    data[1]['country'] = data[0]
    all_countries = all_countries.append(data[1])
all_countries['country'].unique()
df_list

[('DE',
            video_id trending_date  \
  0      LgVi6y5QIjM      17.14.11   
  1      Bayt7uQith4      17.14.11   
  2      1ZAPwfrtAFY      17.14.11   
  3      AHtypnRk7JE      17.14.11   
  4      ZJ9We4bjcg0      17.14.11   
  ...            ...           ...   
  40835  fn5WNxy-Wcw      18.14.06   
  40836  zAFv43lxqHE      18.14.06   
  40837  zSXG5I6Y2fA      18.14.06   
  40838  5d115sePmaU      18.14.06   
  40839  go-F6xvezAM      18.14.06   
  
                                                     title  \
  0      Sing zu Ende! | Gesangseinlagen vom Feinsten |...   
  1      Kinder ferngesteuert im Kiosk! Erwachsene abzo...   
  2      The Trump Presidency: Last Week Tonight with J...   
  3                                    Das Fermi-Paradoxon   
  4               18 SONGS mit Kelly MissesVlog (Sing-off)   
  ...                                                  ...   
  40835  KINGDOM HEARTS III â E3 2018 Pirates of the ...   
  40836                              

In [5]:
# Verifying that every country is in the new dataframe

all_countries['country'].unique()

array(['CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU', 'US'],
      dtype=object)

In [6]:
# Reviewing DataFrame columns

all_countries.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description', 'country'],
      dtype='object')

In [7]:
# Checking number of records at the DataFrame

len(all_countries)

375942

In [8]:
# The dataframe has many redundant columns, 
video_df = all_countries[['video_id','title','channel_title','category_id','publish_time','thumbnail_link','comments_disabled','ratings_disabled','video_error_or_removed','tags','description']]

video_df.head()

,video_id,title,channel_title,category_id,publish_time,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,tags,description
0,n1WpP7iowLc,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,EminemVEVO,10,2017-11-10T17:00:03.000Z,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",Eminem's new track Walk on Water ft. BeyoncÃ© ...
1,0dBIkQ4Mz1M,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...
3,d380meD0W0M,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",ð§: https://ad.gt/yt-perfect\nð°: https://...


In [9]:
# DF of video information
video_df.drop_duplicates(subset='video_id',inplace=True)
video_df

C:\Users\Jose.Vazquez\Anaconda3P\envs\pythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,video_id,title,channel_title,category_id,publish_time,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,tags,description
0,n1WpP7iowLc,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,EminemVEVO,10,2017-11-10T17:00:03.000Z,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",Eminem's new track Walk on Water ft. BeyoncÃ© ...
1,0dBIkQ4Mz1M,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...
3,d380meD0W0M,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",ð§: https://ad.gt/yt-perfect\nð°: https://...
...,...,...,...,...,...,...,...,...,...,...,...
40567,DjLr06pne6Q,Jorja Smith Gets Ready for Bed | Beauty Secret...,Vogue,26,2018-06-12T13:48:29.000Z,https://i.ytimg.com/vi/DjLr06pne6Q/default.jpg,False,False,False,"beauty|""beauty secrets""|""celebrity""|""celebrity...","âItâs not about looking cute,â says the ..."
40751,8O-ZUXGcuWU,Nintendo @ E3 2018: Day 2,Nintendo,20,2018-06-14T01:31:53.000Z,https://i.ytimg.com/vi/8O-ZUXGcuWU/default.jpg,False,False,False,"nintendo|""play""|""play nintendo""|""game""|""gamepl...",ESRB Rating: RATING PENDING to TEEN. Rating Pe...
40752,qPEIPu0V8GQ,Fortnite Celebrity Pro-Am | #FortniteProAm,Fortnite,20,2018-06-13T03:27:30.000Z,https://i.ytimg.com/vi/qPEIPu0V8GQ/default.jpg,False,False,False,"Fortnite|""Epic Games""|""PC""|""PS4""|""Xbox One""|""B...",50 gamers and 50 celebrities will pair up to b...
40760,dS5Thrl-4Kc,CRAYOLA MAKEUP | HIT OR MISS?,Laura Lee,26,2018-06-12T18:55:26.000Z,https://i.ytimg.com/vi/dS5Thrl-4Kc/default.jpg,False,False,False,"Laura88Lee|""crayola""|""crayon makeup""|""crayola ...","Hey Larlees, todays video is me testing Crayol..."


In [10]:
# DF of the trending dates with its information
all_countries=all_countries[['video_id','trending_date','views','likes','dislikes','comment_count','country']]
all_countries

,video_id,trending_date,views,likes,dislikes,comment_count,country
0,n1WpP7iowLc,17.14.11,17158579,787425,43420,125882,CA
1,0dBIkQ4Mz1M,17.14.11,1014651,127794,1688,13030,CA
2,5qpjK5DgCt4,17.14.11,3191434,146035,5339,8181,CA
3,d380meD0W0M,17.14.11,2095828,132239,1989,17518,CA
4,2Vv-BfVoq4g,17.14.11,33523622,1634130,21082,85067,CA
...,...,...,...,...,...,...,...
40944,BZt0qjTWNhw,18.14.06,1685609,38160,1385,2657,US
40945,1h7KV2sjUWY,18.14.06,1064798,60008,382,3936,US
40946,D6Oy4LfoqsU,18.14.06,1066451,48068,1032,3992,US
40947,oV0zkMe1K8s,18.14.06,5660813,192957,2846,13088,US


### 3. Loading data to MongoDB

Creating the connection to MongoDB. The name of the DataBase to be used is **youtube_db**

In [11]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Declare the database
db = client.youtube_db

Declaring the **popular_videos** collection and uploading data from **video_df** DataFrame

In [12]:
# Declare the collection
videos = db.popular_videos
# Resetting index to convert properly to Dictionary
video_df.reset_index(inplace = True)

In [13]:
# Insert the dataframes in the MongoDB
# Insert document into collection
videos.insert_many(video_df.to_dict('records'))

In [ ]:
# Veryfing that imported data

results = videos.find()
for result in results[:3]:
    print(result)

Declaring the **country_records** collection and uploading data from **all_countries** DataFrame

In [15]:
# Declaring the collection for countries
countries = db.country_records

In [17]:
# Resetting index to convert properly to Dictionary
all_countries.reset_index(inplace=True)

In [18]:
# Inserting Document into the collection
countries.insert_many(all_countries.to_dict('records'))

In [19]:
# Veryfing that imported data

results1 = countries.find()
for result in results1[:3]:
    print(result)

{'_id': ObjectId('606e52398278871b9b561318'), 'level_0': 0, 'index': 0, 'video_id': 'n1WpP7iowLc', 'trending_date': '17.14.11', 'views': 17158579, 'likes': 787425, 'dislikes': 43420, 'comment_count': 125882, 'country': 'CA'}
{'_id': ObjectId('606e52398278871b9b561319'), 'level_0': 1, 'index': 1, 'video_id': '0dBIkQ4Mz1M', 'trending_date': '17.14.11', 'views': 1014651, 'likes': 127794, 'dislikes': 1688, 'comment_count': 13030, 'country': 'CA'}
{'_id': ObjectId('606e52398278871b9b56131a'), 'level_0': 2, 'index': 2, 'video_id': '5qpjK5DgCt4', 'trending_date': '17.14.11', 'views': 3191434, 'likes': 146035, 'dislikes': 5339, 'comment_count': 8181, 'country': 'CA'}
